In [21]:
from jinja2 import Template
from datetime import datetime, timedelta
import pandas as pd

In [22]:
with open("template.html") as f:
	template = Template(f.read())

In [23]:
today = datetime.now()

In [24]:
rankings_table = pd.DataFrame(
    {
        "": ["📈", "📉", ""],
        "Rank": ["1st", "2nd", "3rd"],
        "Name": ["Martin", "Lucas", "Krish"],
        "Score": ["96.1%", "34.2%", "31.2%"],
        "Perfect": ["1", "2", "1"],
    }
).to_html(index=False, classes="dataframe", border=0)

In [25]:
report_html = template.render(
	medal="🥇 ",
	name="Martin",
	target_date=today.strftime("%d %B %Y"),
	rankings_table=rankings_table,
    abs_diff=9,
    perf_msg="New York Knicks 1st",
    worst_msg="Minnesota Timberwolves (-6)",
    position="1st",
)

In [26]:
with open("test.html", "w") as f:
	f.write(report_html)

In [28]:
metrics = pd.read_csv("data/metrics.csv")

In [29]:
metrics

,date,name,total_diff,total_perf,worst_tms,worst_by,worst_bys,perfect_tms,perfect_pos,rank
0,2025-08-22,Alexis,80,2,Sunderland,15,-15,Brighton_Crystal Palace,9_12,1
1,2025-08-22,Martin,86,2,Sunderland,17,-17,Manchester City_Brighton,1_9,2
2,2025-08-22,Thomas,92,3,Sunderland,17,-17,Manchester City_Liverpool_Everton,1_4_14,3
3,2025-08-26,Alexis,100,3,Sunderland,10,-10,Arsenal_Chelsea_Wolves,1_4_19,1
4,2025-08-26,Martin,102,2,Sunderland,12,-12,Chelsea_Fulham,4_13,2
...,...,...,...,...,...,...,...,...,...,...
160,2025-10-17,Martin,72,1,Bournemouth,11,-11,Liverpool,2,2
161,2025-10-17,Thomas,82,1,Sunderland,10,-10,Brentford,16,3
162,2025-10-18,Alexis,72,3,Bournemouth,10,-10,Arsenal_Liverpool_Brentford,1_2_16,1
163,2025-10-18,Martin,72,1,Bournemouth,11,-11,Liverpool,2,2


In [32]:
import numpy as np

In [33]:
league = "prem"

today = datetime.now().date()
last_week = today - timedelta(days=7)

metrics = pd.read_csv(f'data/metrics.csv')

metrics_last_week = metrics[metrics['date'].astype(str) == datetime.strftime(last_week, '%Y-%m-%d')][['name', 'rank']]
metrics = metrics[metrics['date'].astype(str) == datetime.strftime(today, '%Y-%m-%d')]

metrics = metrics.merge(
    metrics_last_week,
    on='name',
    how='left',
    suffixes=('', '_past')
)
metrics['trend'] = np.sign(metrics['rank_past'] - metrics['rank'])


In [35]:
from src.utils import int_to_rank

In [36]:
metrics["Rank"] = metrics["rank"].apply(int_to_rank)

metrics[["trend", "Rank", "name", "total_diff", "total_perf"]].rename(columns={
    "trend": "",
    "name": "Name",
    "total_diff": "Score",
    "total_perf": "Perfect",
})

,,Rank,Name,Score,Perfect
0,0,1st,Alexis,72,3
1,0,2nd,Martin,72,1
2,0,3rd,Thomas,82,1


In [1]:
from src.scrape import NBAScraper
from src.eval import NBAEvaluator

In [2]:
standings = NBAScraper().scrape_standings()

In [3]:
standings["Rk"] = list(range(1, 16)) + list(range(1, 16))

In [5]:
NBAEvaluator(standings).evaluate(names=["Martin", "Lucas"]).to_csv("data/nba/metrics.csv", index=False)

In [11]:
from src.eval import eval_preds
import os
import pandas as pd

In [66]:
prediction_folder = "predictions/nba"

metrics_list = []
names = ["Martin", "Lucas"]
for conf in ["East", "West"]:

    conf_standings = standings.copy()
    conf_standings = conf_standings[
        conf_standings["Conference"] == conf
    ]

    res_list = []
    for name in names:

        prediction_path = os.path.join(prediction_folder, conf.lower(), f"{name.lower()}.txt")
        with open(prediction_path, "r") as f:
            preds = f.read().splitlines()

        res_list.append(
            eval_preds(conf_standings, preds)
        )

    conf_metrics = pd.concat(res_list)
    conf_metrics.insert(0, "name", names)
    conf_metrics["conference"] = conf
    metrics_list.append(conf_metrics)

    all_metrics = pd.concat(metrics_list)
    metrics = all_metrics.groupby("name").apply(compute_overall_metrics, include_groups=False).droplevel(1).reset_index()
    metrics["conference"] = "Overall"
    metrics = pd.concat([all_metrics, metrics], ignore_index=True)

In [ ]:
metrics["rank"] = metrics.groupby("conference")["spearmanr"].rank(method="min")

,name,spearmanr,total_diff,total_perf,worst_tms,worst_by,worst_bys,perfect_tms,perfect_pos,conference
0,Martin,0.407143,54,3,BOS,9,-9,NYK_ATL_CHI,1_5_12,East
1,Lucas,0.175000,66,2,BOS_WAS,10,-10_10,NYK_MIA,1_4,East
2,Martin,0.285714,58,1,UTA,13,-13,SAS,9,West
3,Lucas,0.217857,60,3,MIN,12,-12,SAS_SAC_PHO,9_11_15,West
4,Lucas,0.196429,126,5,MIN,12,-12,NYK_MIA_SAS_SAC_PHO,1_4_9_11_15,Overall
5,Martin,0.346429,112,4,UTA,13,-13,NYK_ATL_CHI_SAS,1_5_12_9,Overall


In [73]:
metrics["rank"] = metrics.sort_values(["spearmanr", "total_perf", "total_diff", "worst_by"], ascending=[False, False, True, True]).groupby("conference").cumcount() + 1

In [74]:
metrics

,name,spearmanr,total_diff,total_perf,worst_tms,worst_by,worst_bys,perfect_tms,perfect_pos,conference,rank
0,Martin,0.407143,54,3,BOS,9,-9,NYK_ATL_CHI,1_5_12,East,1
1,Lucas,0.175000,66,2,BOS_WAS,10,-10_10,NYK_MIA,1_4,East,2
2,Martin,0.285714,58,1,UTA,13,-13,SAS,9,West,1
3,Lucas,0.217857,60,3,MIN,12,-12,SAS_SAC_PHO,9_11_15,West,2
4,Lucas,0.196429,126,5,MIN,12,-12,NYK_MIA_SAS_SAC_PHO,1_4_9_11_15,Overall,2
5,Martin,0.346429,112,4,UTA,13,-13,NYK_ATL_CHI_SAS,1_5_12_9,Overall,1


In [63]:
all_metrics = pd.concat(metrics_list)


def compute_overall_metrics(metrics: pd.DataFrame) -> pd.DataFrame:

    worsts = metrics[metrics["worst_by"] == metrics["worst_by"].max()].copy()
    return pd.DataFrame(
        {
            "spearmanr": [metrics["spearmanr"].mean()],
            "total_diff": [metrics["total_diff"].sum()],
            "total_perf": [metrics["total_perf"].sum()],
            "worst_tms": ["_".join(worsts["worst_tms"])],
            "worst_by": [metrics["worst_by"].max()],
            "worst_bys": ["_".join(worsts["worst_bys"])],
            "perfect_tms": ["_".join(metrics["perfect_tms"]).strip("_")],
            "perfect_pos": ["_".join(metrics["perfect_pos"]).strip("_")],
        }
    ).reset_index(drop=True)

In [59]:
all_metrics.groupby("name").apply(overall_metrics, include_groups=False).droplevel(1).reset_index()

,name,spearmanr,total_diff,total_perf,worst_tms,worst_by,worst_bys,perfect_tms,perfect_pos
0,Lucas,0.196429,126,5,MIN,12,-12,NYK_MIA_SAS_SAC_PHO,1_4_9_11_15
1,Martin,0.346429,112,4,UTA,13,-13,NYK_ATL_CHI_SAS,1_5_12_9


In [48]:
all_metrics

,name,spearmanr,total_diff,total_perf,worst_tms,worst_by,worst_bys,perfect_tms,perfect_pos,conference
0,Martin,0.407143,54,3,BOS,9,-9,NYK_ATL_CHI,1_5_12,East
0,Lucas,0.175000,66,2,BOS_WAS,10,-10_10,NYK_MIA,1_4,East
0,Martin,0.285714,58,1,UTA,13,-13,SAS,9,West
0,Lucas,0.217857,60,3,MIN,12,-12,SAS_SAC_PHO,9_11_15,West


In [6]:
from datetime import datetime
import pandas as pd
import os

from src.ref import PLAYERS
from src.scrape import get_scraper
from src.eval import get_evaluator

In [31]:

league = os.getenv("LEAGUE", "nba")


today = datetime.now().date()

names = PLAYERS[league]

scraper = get_scraper(league)
standings = scraper.scrape_standings()


# standings["Rk"] = list(range(1, 16)) + list(range(1, 16))
# standings[["Rk", "Team", "Conference", "Date"]].to_csv(f"data/{league}/standings.csv", index=False)

In [ ]:
evaluator = get_evaluator(league, standings)

metrics = evaluator.evaluate(names)

metrics.insert(0, "date", today)
if league == "prem":
    metrics.drop(columns="spearmanr", inplace=True)

metrics.to_csv(f"data/{league}/metrics.csv", index=False, header=False, mode="a")

In [19]:
metrics.to_csv(f"data/{league}/metrics.csv", index=False, mode="a", header=False)

In [17]:
pd.read_csv(f"data/{league}/metrics.csv").columns

Index(['date', 'name', 'total_diff', 'total_perf', 'worst_tms', 'worst_by',
       'worst_bys', 'perfect_tms', 'perfect_pos', 'rank'],
      dtype='object')

In [1]:
from datetime import datetime
import pandas as pd
import os

from src.ref import PLAYERS
from src.scrape import get_scraper
from src.eval import get_evaluator


league = os.getenv("LEAGUE", "nba")


today = datetime.now().date()

names = PLAYERS[league]

scraper = get_scraper(league)
standings = scraper.scrape_standings()
standings.to_csv(f"data/{league}/standings.csv", index=False, header=False, mode="a")


standings["Rk"] = list(range(1, 16)) + list(range(1, 16))


evaluator = get_evaluator(league, standings)

metrics = evaluator.evaluate(names)
metrics.insert(0, "date", today)

if league == "prem":
    metrics.drop(columns="spearmanr", inplace=True)

metrics.to_csv(f"data/{league}/metrics.csv", index=False, header=False, mode="a")

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from src.comms import format_rankings_msg, format_whatsapp_msg
from src.ref import EMAILS


league = "nba"


SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
USERNAME = os.getenv('GMAIL_USERNAME')
PASSWORD = os.getenv('GMAIL_APP_PASSWORD')


today = datetime.now().date()
yesterday = today - timedelta(days=1)

metrics = pd.read_csv(f'data/{league}/metrics.csv')
metrics = metrics[metrics["conference"] == "Overall"].copy()

metrics_yesterday = metrics[metrics['date'].astype(str) == datetime.strftime(yesterday, '%Y-%m-%d')][['name', 'rank']]
metrics = metrics[metrics['date'].astype(str) == datetime.strftime(today, '%Y-%m-%d')]

metrics = metrics.merge(
    metrics_yesterday,
    on='name',
    how='left',
    suffixes=('', '_past')
)
metrics['trend'] = np.sign(metrics['rank_past'] - metrics['rank']).fillna(0)

In [9]:
rankings_html = metrics[["trend", "rank", "name", "total_diff", "total_perf"]].rename(
    columns={
        "trend": "",
        "name": "Name",
        "total_diff": "Score",
        "total_perf": "Perfect",
    }
).sort_values("rank").to_html(index=False, classes="dataframe", border=0)

In [28]:
from src.utils import int_to_rank
from jinja2 import Template
import json

In [46]:
subject = f'{league.upper()} Predictions Weekly Update  -  {today.strftime("%d %b %Y")}'

with open("template.html") as f:
    template = Template(f.read())


with open("utils/abbr_nba.json", "r") as f:
    abbr_nba = json.load(f)


for _, row in metrics.sort_values('name').iterrows():

    name = row['name']
    rank = row['rank']
    medal = {1: '🥇', 2: '🥈', 3: '🥉'}.get(rank, '')
    position = int_to_rank(rank)


    score = (row['spearmanr'] + 1) / 2

    to_email = EMAILS.get(name)

    perf_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({int_to_rank(pos)})"
            for tm, pos in zip(
                row['perfect_tms'].split("_"),
                row['perfect_pos'].split("_"),
            )
        ]
    )

    worst_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({pos})"
            for tm, pos in zip(
                row['worst_tms'].split("_"),
                row['worst_bys'].split("_"),
            )
        ]
    )


    report_html = template.render(
        medal=medal,
        name=name,
        report_date=today.strftime("%d %B %Y"),
        rankings_table=rankings_html,
        score=f"{score:.1%}",
        abs_diff=row['total_diff'],
        perf_msg=perf_msg,
        worst_msg=worst_msg,
        position=position,
    )

    break


    # msg = MIMEMultipart()
    # msg['From'] = USERNAME
    # msg['To'] = to_email
    # msg['Subject'] = subject

    # msg.attach(MIMEText(body, 'plain'))

    # # Send the email
    # try:
    #     server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    #     server.starttls()
    #     server.login(USERNAME, PASSWORD)
    #     server.send_message(msg)
    #     print(f'Email sent successfully to {to_email}!')
    # except Exception as e:
    #     print(f"Failed to send email base of '{e}'")
    # finally:
    #     server.quit()
    #     time.sleep(.1)

In [47]:
report_html

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8" />\n  <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  <title>Predictions Email</title>\n  <style>\n    body {\n      font-family: \'Radikal\', Arial, sans-serif;\n      background-color: #C8102E;\n      margin: 0;\n      padding: 0;\n      color: #333;\n    }\n    .email-container {\n      max-width: 600px;\n      margin: 30px auto;\n      background-color: #FFFFFF;\n      border-radius: 20px;\n      overflow: hidden;\n      border: solid 2px;\n      border-color: #1D428A;\n    }\n    .header {\n      background-color: #1D428A;\n      color: #ffffff;\n      padding: 30px 30px;\n      text-align: center;\n    }\n    .header h1 {\n      margin: 0;\n      font-size: 24px;\n      letter-spacing: 0.5px;\n    }\n    .content {\n      padding: 30px;\n    }\n    .ranking {\n      font-size: 18px;\n      font-weight: bold;\n      color: #1D428A;\n      margin-bottom: 40px;\n    }\n    .summary {\n   

In [48]:
with open("test.html", "w") as f:
    f.write(report_html)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import time
import os
import sys
import json
from jinja2 import Template
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from src.ref import EMAILS, VARS
from src.utils import int_to_rank


league = "nba"

REF_DATE = VARS[league]["ref_date"]
EMAIL_FREQUENCY = VARS[league]["email_frequency"]

today = datetime.now().date() - timedelta(days=1)
yesterday = today - timedelta(days=1)

metrics = pd.read_csv(f'data/{league}/metrics.csv')
metrics = metrics[metrics["conference"] == "Overall"].copy()

metrics_yesterday = metrics[metrics['date'].astype(str) == datetime.strftime(yesterday, '%Y-%m-%d')][['name', 'rank']]
metrics = metrics[metrics['date'].astype(str) == datetime.strftime(today, '%Y-%m-%d')]

metrics = metrics.merge(
    metrics_yesterday,
    on='name',
    how='left',
    suffixes=('', '_past')
)
metrics['trend'] = (metrics['rank_past'] - metrics['rank']).fillna(0)

metrics["Rank"] = metrics.apply(lambda row: f"{int_to_rank(row['rank'])}", axis=1)
metrics["Score"] = metrics['spearmanr'].apply(lambda x: f"{100 * (x + 1) / 2:.1f}%")


rankings_html = metrics[["rank", "Rank", "name", "Score", "total_perf"]].rename(
    columns={
        "name": "Name",
        "Rank": "",
        "total_perf": "Perfect",
    }
).sort_values("rank").drop(columns="rank").to_html(index=False, classes="dataframe", border=0)

subject = f'{league.upper()} Predictions Update  -  {today.strftime("%d %b %Y")}'


In [8]:

rankings_html = metrics[["rank", "Rank", "name", "Score", "total_perf"]].rename(
    columns={
        "name": "Name",
        "Rank": "",
        "total_perf": "Perfect",
    }
).sort_values("rank").drop(columns="rank").to_html(index=False, classes="dataframe", border=0)

subject = f'{league.upper()} Predictions Update  -  {today.strftime("%d %b %Y")}'


In [9]:

with open("template.html") as f:
    template = Template(f.read())

with open("utils/abbr_nba.json", "r") as f:
    abbr_nba = json.load(f)


for _, row in metrics.sort_values('name').iterrows():

    name = row['name']
    rank = row['rank']
    medal = {1: '🥇', 2: '🥈', 3: '🥉'}.get(rank, '')
    position = int_to_rank(rank)


    score = (row['spearmanr'] + 1) / 2

    perf_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({int_to_rank(pos)})"
            for tm, pos in zip(
                row['perfect_tms'].split("_"),
                row['perfect_pos'].split("_"),
            )
        ]
    )

    worst_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({pos})"
            for tm, pos in zip(
                row['worst_tms'].split("_"),
                row['worst_bys'].split("_"),
            )
        ]
    )

    report_html = template.render(
        medal=medal,
        name=name,
        report_date=today.strftime("%d %B %Y"),
        rankings_table=rankings_html,
        score=f"{score:.1%}",
        abs_diff=row['total_diff'],
        perf_msg=perf_msg,
        worst_msg=worst_msg,
        position=position,
    )

    break


with open("test.html", "w") as f:
    f.write(report_html)

In [15]:
url = "https://www.espn.com/nba/standings"

In [16]:
from bs4 import BeautifulSoup
import requests

In [19]:
import json

In [20]:
with open("utils/abbr_nba.json", "r") as f:
    abbr_dict = json.load(f)
abbr_map = {v: k for k, v in abbr_dict.items()}

In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO

url = "https://sports.yahoo.com/nba/standings/?selectedTab=CONFERENCE"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
tables = soup.find_all("table")

conf_standings = []
for table in tables[-2:]:

    df = pd.read_html(StringIO(str(table)))[0]
    conf = df.columns[0]

    standings = df[[conf]].rename(columns={conf: "Team"})
    standings["Team"] = standings["Team"].map(abbr_map)
    standings["Conference"] = conf.removesuffix("ern")
    standings["Rk"] = range(1, 16)
    standings["Date"] = "date"
    conf_standings.append(
        standings[["Conference", "Rk", "Team", "Date"]]
    )

pd.concat(conf_standings, ignore_index=True)

,Conference,Rk,Team,Date
0,East,1,CHI,date
1,East,2,ORL,date
2,East,3,PHI,date
3,East,4,TOR,date
4,East,5,CHA,date
5,East,6,MIL,date
6,East,7,NYK,date
7,East,8,BOS,date
8,East,9,DET,date
9,East,10,MIA,date


In [8]:
table = tables[-1]

In [9]:
from io import StringIO

In [13]:
df = pd.read_html(StringIO(str(table)))[0]
conf = df.columns[0]

standings = df[[conf]].rename(columns={conf: "Team"})
standings["Conference"] = conf.removesuffix("ern")
standings["Rk"] = range(1, 16)
standings["Date"] = "date"
standings[["Conference", "Rk", "Team", "Date"]]

,Conference,Rk,Team,Date
0,West,1,San Antonio,date
1,West,2,Golden State,date
2,West,3,Minnesota,date
3,West,4,Utah,date
4,West,5,Memphis,date
5,West,6,Phoenix,date
6,West,7,Oklahoma City,date
7,West,8,Houston,date
8,West,9,Portland,date
9,West,10,Sacramento,date


In [54]:
conf = df.columns[0]

In [47]:
from src.scrape import get_scraper

scraper = get_scraper("nba")
standings = scraper.scrape_standings()

In [48]:
standings

,Conference,Rk,Team,Date
0,East,1,CHI,2025-10-23
1,East,2,ORL,2025-10-23
2,East,3,PHI,2025-10-23
3,East,4,TOR,2025-10-23
4,East,5,CHA,2025-10-23
5,East,6,MIL,2025-10-23
6,East,7,NYK,2025-10-23
7,East,8,BOS,2025-10-23
8,East,9,DET,2025-10-23
9,East,10,MIA,2025-10-23


In [24]:
from src.scrape import get_scraper

In [26]:
get_scraper("nba").scrape_standings()

,Conference,Rk,Team,Date
0,East,1,CHI,2025-10-23
1,East,2,ORL,2025-10-23
2,East,3,PHI,2025-10-23
3,East,4,TOR,2025-10-23
4,East,5,CHA,2025-10-23
5,East,6,MIL,2025-10-23
6,East,7,NYK,2025-10-23
7,East,8,BOS,2025-10-23
8,East,9,DET,2025-10-23
9,East,10,MIA,2025-10-23


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import time
import os
import sys
import json
from jinja2 import Template
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from src.ref import EMAILS, VARS
from src.utils import int_to_rank


league = "nba"

REF_DATE = VARS[league]["ref_date"]
EMAIL_FREQUENCY = VARS[league]["email_frequency"]

today = datetime.now().date()
yesterday = today - timedelta(days=1)

# if (today - REF_DATE).days % EMAIL_FREQUENCY != 0:
#     print("No email to send today.")
#     sys.exit()


SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
USERNAME = os.getenv('NBA_GMAIL_USERNAME')
PASSWORD = os.getenv('NBA_GMAIL_APP_PASSWORD')


metrics = pd.read_csv(f'data/{league}/metrics.csv')
metrics = metrics[metrics["conference"] == "Overall"].copy()

metrics_yesterday = metrics[metrics['date'].astype(str) == datetime.strftime(yesterday, '%Y-%m-%d')][['name', 'rank']]
metrics = metrics[metrics['date'].astype(str) == datetime.strftime(today, '%Y-%m-%d')]

metrics = metrics.merge(
    metrics_yesterday,
    on='name',
    how='left',
    suffixes=('', '_past')
)
metrics['trend'] = (metrics['rank_past'] - metrics['rank']).fillna(0)

metrics["Rank"] = metrics.apply(lambda row: f"{int_to_rank(row['rank'])}", axis=1)
metrics["Score"] = metrics['spearmanr'].apply(lambda x: f"{100 * (x + 1) / 2:.1f}%")

In [2]:

rankings_html = metrics[["rank", "Rank", "name", "Score", "total_perf"]].rename(
    columns={
        "name": "Name",
        "Rank": "",
        "total_perf": "Perfect",
    }
).sort_values("rank").drop(columns="rank").to_html(index=False, classes="dataframe", border=0)

subject = f'{league.upper()} Predictions Update  -  {today.strftime("%d %b %Y")}'

with open("template.html") as f:
    template = Template(f.read())


with open("utils/abbr_nba.json", "r") as f:
    abbr_nba = json.load(f)

In [3]:

for _, row in metrics.sort_values('name').iterrows():

    name = row['name']
    rank = row['rank']
    medal = {1: '🥇', 2: '🥈', 3: '🥉'}.get(rank, '')
    position = int_to_rank(rank)

    # to_email = EMAILS.get(name)
    to_email = "martinbog19@gmail.com"

    perf_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({int_to_rank(pos)})"
            for tm, pos in zip(
                row['perfect_tms'].split("_"),
                row['perfect_pos'].split("_"),
            )
        ]
    ) if row['perfect_tms'] != "" else "None..."

    worst_msg = ", ".join(
        [
            f"{abbr_nba[tm]} ({pos})"
            for tm, pos in zip(
                row['worst_tms'].split("_"),
                row['worst_bys'].split("_"),
            )
        ]
    )

    report_html = template.render(
        medal=medal,
        name=name,
        report_date=today.strftime("%d %B %Y"),
        rankings_table=rankings_html,
        score=row["Score"],
        abs_diff=row['total_diff'],
        perf_msg=perf_msg,
        worst_msg=worst_msg,
        position=position,
    )

    email = MIMEMultipart()
    email['From'] = USERNAME
    email['To'] = to_email
    email['Subject'] = subject

    email.attach(MIMEText(report_html, 'html'))

    # Send the email
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(USERNAME, PASSWORD)
        server.send_message(email)
        print(f'Email sent successfully to {to_email}!')
    except Exception as e:
        print(f"Failed to send email to {to_email} because of '{e}'")
    finally:
        server.quit()
        time.sleep(1)


    break


Email sent successfully to martinbog19@gmail.com!
